In [ ]:
import json

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# from umap import UMAP
from sklearn.decomposition import PCA

from sklearn.model_selection import train_test_split, StratifiedShuffleSplit, GroupShuffleSplit

from sklearn.neighbors import KNeighborsClassifier
from sklearn.neural_network import MLPClassifier

In [ ]:
from source.constants import RANDOM_SEED
from source.constants import DATA_DIR, FEATURE_VECTORS_SAVE_DIR, ANNOTATIONS_SAVE_DIR
from source.constants import ALL_TISSUE_TYPES, ALL_CANCER_TYPES, ALL_IMG_NORMS, ALL_EXTRACTOR_MODELS, ALL_DIMENSIONALITY_REDUCTION_METHODS, ALL_CLUSTERING_ALGORITHMS, ALL_DISTANCE_METRICS
from source.constants import ORIGINAL_2_PRETTY_MODEL_NAMES

print(f"DATA_DIR: {DATA_DIR}")
print(f"FEATURE_VECTORS_SAVE_DIR: {FEATURE_VECTORS_SAVE_DIR}")
print(f"ANNOTATIONS_SAVE_DIR: {ANNOTATIONS_SAVE_DIR}")

print("ALL_TISSUE_TYPES:", ALL_TISSUE_TYPES)
print("ALL_CANCER_TYPES:", ALL_CANCER_TYPES)
print("ALL_EXTRACTOR_MODELS:", ALL_EXTRACTOR_MODELS)
print("ALL_IMG_NORMS:", ALL_IMG_NORMS)
print("ALL_DIMENSIONALITY_REDUCTION_METHODS:",
      ALL_DIMENSIONALITY_REDUCTION_METHODS)
print("ALL_CLUSTERING_ALGORITHMS:", ALL_CLUSTERING_ALGORITHMS)
print("ALL_DISTANCE_METRICS:", ALL_DISTANCE_METRICS)

In [ ]:
# TODO: Set the cancer type, extractor name, and image normalization method

# CANCER_TYPES = ['lung_aca', 'lung_n', 'lung_scc']
# CANCER_TYPES = ['colon_aca', 'colon_n']
CANCER_TYPES = ['lung_aca', 'lung_n', 'lung_scc', 'colon_aca', 'colon_n']

# should be the same for every extractor for comparability
IMG_NORM = 'imagenet'
# IMG_NORM = 'resize_only'

EXTRACTOR_NAME = 'imagenet_resnet18-last-layer' # worst
# EXTRACTOR_NAME = 'owkin-phikon'               # medium
# EXTRACTOR_NAME = 'UNI'                        # best

# use validation set and early stopping, or just use train set until loss convergence
use_val_set = False
# decrease training size twice each time
test_sizes = [0.2, 0.8, 0.95]
# cross-validation splits
n_splits = 10

assert set(CANCER_TYPES).issubset(set(ALL_CANCER_TYPES))
assert EXTRACTOR_NAME in ALL_EXTRACTOR_MODELS
assert IMG_NORM in ALL_IMG_NORMS

In [ ]:
def get_knn():
    return KNeighborsClassifier(n_neighbors=1)


def get_mlp(use_val_set):
    if use_val_set:
        return MLPClassifier(random_state=RANDOM_SEED, hidden_layer_sizes=(), max_iter=1000, early_stopping=True, validation_fraction=0.2)
    else:
        return MLPClassifier(random_state=RANDOM_SEED, hidden_layer_sizes=(), max_iter=1000)

## Load the features, labels, and clusters

In [ ]:
def get_sorted_cluster_labels(annotations_df, ids_2_imgpaths):
    assert len(set(ids_2_imgpaths.values())) == len(list(ids_2_imgpaths.values())
                                                    ), "Can only reverse a bijective mapping, duplicate values found."
    img_paths_2_int_ids = {v: int(k) for k, v in ids_2_imgpaths.items()}

    cluster_labels = annotations_df['cluster_label'].values.astype(int)
    img_paths = annotations_df['img_path'].values
    img_paths_2_cluster_labels = dict(zip(img_paths, cluster_labels))

    assert set(img_paths_2_int_ids.keys()) == set(img_paths_2_cluster_labels.keys(
    )), "The img_paths in the annotations_df must be the same as the img_paths in the ids_2_imgpaths dictionary."

    intids_2_cluster_labels = {
        img_paths_2_int_ids[img_path]: img_paths_2_cluster_labels[img_path]
        for img_path in img_paths_2_cluster_labels.keys()
    }

    cluster_labels_sorted_by_intids = [intids_2_cluster_labels[i] for i in sorted(intids_2_cluster_labels.keys())]
    cluster_labels_sorted_by_intids = np.array(cluster_labels_sorted_by_intids)

    assert all(np.unique(cluster_labels_sorted_by_intids) == np.arange(cluster_labels_sorted_by_intids.max()+1))

    return cluster_labels_sorted_by_intids

In [ ]:
X = []
y = []
all_cluster_labels = []

current_max_plus_one = 0
for i, cancer_type in enumerate(CANCER_TYPES):
    feature_path = f"{FEATURE_VECTORS_SAVE_DIR}/{cancer_type}/{EXTRACTOR_NAME}/{IMG_NORM}/features.npy"
    with open(f"{FEATURE_VECTORS_SAVE_DIR}/{cancer_type}/{EXTRACTOR_NAME}/{IMG_NORM}/ids_2_img_paths.json", 'r') as f:
        ids_2_imgpaths = json.load(f)
    annotations_path = f"{ANNOTATIONS_SAVE_DIR}/{cancer_type}/UNI/resize_only/final_clusters.csv"
    features_arr = np.load(feature_path)
    labels = np.full(features_arr.shape[0], fill_value=i)
    X.append(features_arr)
    y.append(labels)

    annotations_df = pd.read_csv(annotations_path)
    sorted_cluster_labels = get_sorted_cluster_labels(annotations_df, ids_2_imgpaths)
    print(f"cancer type {cancer_type} has {len(np.unique(sorted_cluster_labels))} clusters")
    cancer_cluster_labels = current_max_plus_one + sorted_cluster_labels
    all_cluster_labels.append(cancer_cluster_labels)

    # update the current_max_plus_one
    current_max_plus_one += sorted_cluster_labels.max() + 1

X = np.concatenate(X)
y = np.concatenate(y)
all_cluster_labels = np.concatenate(all_cluster_labels)

# check that currnet_max_plus_one works as expected - i.e. no overlap between the cluster labels of different cancer types
assert len(set(all_cluster_labels[:5000]).intersection(
    set(all_cluster_labels[5000:]))) == 0
assert len(set(all_cluster_labels[:10000]).intersection(
    set(all_cluster_labels[10000:]))) == 0
print("Total groups:", len(set(all_cluster_labels)))

## Classes Visualisation in Feature Space

In [ ]:
dim_reduction = PCA(n_components=2, random_state=RANDOM_SEED)
# dim_reduction = UMAP(n_components=2)

np.random.seed(RANDOM_SEED)
random_order = np.random.permutation(X.shape[0])
X_random_order = X[random_order]
y_random_order = y[random_order]
all_cluster_labels_random_order = all_cluster_labels[random_order]

X_random_order_reduced = dim_reduction.fit_transform(X_random_order)

In [ ]:
# plot X_umap with y as the color

plt.figure(figsize=(5, 5))
sns.scatterplot(
    x=X_random_order_reduced[:, 0],
    y=X_random_order_reduced[:, 1],
    hue=[CANCER_TYPES[i] for i in y_random_order],
    palette='tab20',
    s=10
)
# plt.title(f"Feature vectors: {IMG_NORM} normalisation + {EXTRACTOR_NAME} extraction")
plt.title(ORIGINAL_2_PRETTY_MODEL_NAMES[EXTRACTOR_NAME])
plt.legend(title='Cancer type',)
plt.show()

## Precision@1 and Precision@5 for the original dataset restricted to tissue type

In [ ]:
# from source.eval_utils import precision_at_1, precision_at_k

# y_connectivity_matrix = (y[:, np.newaxis] == y[np.newaxis, :]).astype(int)
# plt.imshow(y_connectivity_matrix, cmap='gray')
# plt.show()
# print("Precision@1", precision_at_1(X, y_connectivity_matrix))
# print("Precision@5", precision_at_k(X, y_connectivity_matrix, k=5))

## Precision@1 and Precision@5 for the cleaned dataset restricted to tissue type (take 1 example from each cluster)

In [ ]:
all_cluster_labels

# Simple classification of the original dataset, no cross-validation

In [ ]:
knn = get_knn()
mlp = get_mlp(use_val_set)

for test_size in test_sizes:
    X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=test_size, stratify=y, random_state=RANDOM_SEED)

    # Fit and score KNN
    knn.fit(X_train, y_train)
    y_pred_knn = knn.predict(X_test)
    score_knn = knn.score(X_test, y_test)
    print(f"KNN - Test size: {test_size:.4}, score: {score_knn:.4}")

    # Fit and score MLP
    mlp.fit(X_train, y_train)
    y_pred_mlp = mlp.predict(X_test)
    score_mlp = mlp.score(X_test, y_test)
    print(f"MLP - Test size: {test_size}, score: {score_mlp:.4}")

    print()

# print("mlp.out_activation_:", mlp.out_activation_)
# print("len(mlp.coefs_):", len(mlp.coefs_))
# for i in range(len(mlp.coefs_)):
#     print(f"\tmlp.coefs_[{i}].shape:", mlp.coefs_[i].shape)

## N-samples Cross-validation of the original LC25000 dataset

In [ ]:
knn = get_knn()
mlp = get_mlp(use_val_set)

original_test_size_2_scores = {}

for test_size in test_sizes:
    knn_scores = []
    mlp_scores = []

    sss = StratifiedShuffleSplit(
        n_splits=n_splits, test_size=test_size, random_state=RANDOM_SEED)

    for train_index, test_index in sss.split(X, y):
        X_train, X_test = X[train_index], X[test_index]
        y_train, y_test = y[train_index], y[test_index]

        # Evaluate KNN
        knn.fit(X_train, y_train)
        score_knn = knn.score(X_test, y_test)
        knn_scores.append(score_knn)

        # Evaluate MLP
        mlp.fit(X_train, y_train)
        score_mlp = mlp.score(X_test, y_test)
        mlp_scores.append(score_mlp)

    original_test_size_2_scores[test_size] = {
        'knn': knn_scores,
        'mlp': mlp_scores,
    }

    print(f"KNN - Test size: {test_size:.4}, mean score: {np.mean(knn_scores):.4}, std: {np.std(knn_scores):.4}")
    print(f"MLP - Test size: {test_size}, mean score: {np.mean(mlp_scores):.4}, std: {np.std(mlp_scores):.4}")
    print()

## N-samples Grouped Cross-validation on LC25000-clean grouped by clusters (prototypes)

In [ ]:
knn = get_knn()
mlp = get_mlp(use_val_set)

clean_test_size_2_scores = {}

for test_size in test_sizes:
    knn_scores = []
    mlp_scores = []

    gss = GroupShuffleSplit(
        n_splits=n_splits, test_size=test_size, random_state=RANDOM_SEED)

    for train_index, test_index in gss.split(X, y, groups=all_cluster_labels):
        X_train, X_test = X[train_index], X[test_index]
        y_train, y_test = y[train_index], y[test_index]
        all_cluster_labels_train, all_cluster_labels_test = all_cluster_labels[train_index], all_cluster_labels[test_index]
        assert len(set(all_cluster_labels_train).intersection(all_cluster_labels_test)) == 0, "Overlap between the cluster labels of the training and test sets"

        # Evaluate KNN
        knn.fit(X_train, y_train)
        score_knn = knn.score(X_test, y_test)
        knn_scores.append(score_knn)

        # Evaluate MLP
        mlp.fit(X_train, y_train)
        score_mlp = mlp.score(X_test, y_test)
        mlp_scores.append(score_mlp)

    clean_test_size_2_scores[test_size] = {
        'knn': knn_scores,
        'mlp': mlp_scores,
    }

    print(f"KNN - Test size: {test_size:.4}, mean score: {np.mean(knn_scores):.4}, std: {np.std(knn_scores):.4}")
    print(f"MLP - Test size: {test_size}, mean score: {np.mean(mlp_scores):.4}, std: {np.std(mlp_scores):.4}")
    print()

    # break

# Combined Results

In [ ]:
all_test_size_2_scores = {
    'original': original_test_size_2_scores,
    'clean': clean_test_size_2_scores,
}
all_test_size_2_scores

In [ ]:
def get_mean_scores(test_size_2_scores):
    return {
        test_size: {
            cls: f"{np.mean(scores[cls]).round(3)} $\pm$ {np.std(scores[cls]).round(3)}"
            for cls in scores.keys()
        }
        for test_size, scores in test_size_2_scores.items()
    }

In [ ]:
print("Image Normalization", IMG_NORM)
print("Extractor Name:", EXTRACTOR_NAME)
print("Using validation set:", use_val_set)

In [ ]:
pd.DataFrame(get_mean_scores(original_test_size_2_scores))

In [ ]:
pd.DataFrame(get_mean_scores(clean_test_size_2_scores))